<a href="https://colab.research.google.com/github/lacey-conrad/WGCNA-work/blob/main/treatment_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split


from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras

from keras import Sequential
from keras import layers
from keras import models
from keras.layers import Dense, Flatten, Conv2D
from keras import Model
from sklearn.decomposition import PCA  # to apply PCA

In [2]:
df_gastroc = pd.read_csv('gastroc_raw_counts.csv')
traits = pd.read_csv('thermo_capacity_traits_all.csv')

In [3]:
#traits.tail(50)

In [4]:
traits = traits.rename(columns={'mouse_id': 'Animal'})

In [5]:
df = pd.DataFrame(traits, columns=['Animal','acclim1'])
#df.tail(50)

In [6]:
#df_gastroc.head()

In [7]:
df_gastroc['Row_Mean'] = df_gastroc.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [8]:
df_gastroc = df_gastroc[df_gastroc.Row_Mean >= 10]

In [9]:
# Keep!!!
df_gastroc.drop("Row_Mean", axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
df_gastroc = df_gastroc.T

In [11]:
df_gastroc.columns = df_gastroc.iloc[0]

In [12]:
df_gastroc = df_gastroc.iloc[1: , :]

In [13]:
df_gastroc.head()

Unnamed: 0,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,Cacng6,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
LN.F1.113.2be,10,27,336,384,6,13,33,83,23,111,...,11886,276,122,92,6169,5420,2144,16304,173,1511
LN.F1.113.aa,12,35,264,340,13,7,51,36,39,184,...,12638,371,207,81,7024,6495,1673,16561,142,1280
LN.F1.113.ae,5,21,175,407,21,10,43,43,30,200,...,9748,272,135,67,5841,5400,1414,13018,103,955
LN.F1.113.ag,6,25,298,446,21,16,54,61,23,131,...,16001,495,230,116,9873,7522,2011,21779,196,1599
LN.F1.113.ak,7,25,425,526,17,19,58,132,51,259,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [14]:
### normalize and merge after this point

In [15]:
scaler = StandardScaler()
scaler.fit(df_gastroc)
X_trans = scaler.transform(df_gastroc)

In [16]:
#Applying PCA
#Taking no. of Principal Components as 3
pca = PCA(n_components = 3)
pca.fit(X_trans)

PCA(n_components=3)

In [17]:
data_pca = pca.transform(X_trans)
data_pca = pd.DataFrame(data_pca,columns=['PC1','PC2','PC3'])
data_pca.head()

,PC1,PC2,PC3
0,-44.117166,26.196616,6.950873
1,-47.573065,-12.572077,-11.710404
2,-72.231403,-6.664231,-6.881561
3,17.838188,-20.307664,3.901166
4,73.311040,2.032320,-22.955084


In [18]:
df_gastroc.head()

Unnamed: 0,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,Cacng6,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
LN.F1.113.2be,10,27,336,384,6,13,33,83,23,111,...,11886,276,122,92,6169,5420,2144,16304,173,1511
LN.F1.113.aa,12,35,264,340,13,7,51,36,39,184,...,12638,371,207,81,7024,6495,1673,16561,142,1280
LN.F1.113.ae,5,21,175,407,21,10,43,43,30,200,...,9748,272,135,67,5841,5400,1414,13018,103,955
LN.F1.113.ag,6,25,298,446,21,16,54,61,23,131,...,16001,495,230,116,9873,7522,2011,21779,196,1599
LN.F1.113.ak,7,25,425,526,17,19,58,132,51,259,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [19]:
#### Normalize and merge before this point

In [20]:
df_gastroc = df_gastroc.reset_index()

In [21]:
df_gastroc = df_gastroc.rename(columns={'index': 'Animal'})

In [22]:
df_gastroc.head()

Unnamed: 0,Animal,LOC102903073,LOC102904318,Ppp2r1a,LOC102906476,LOC102922856,LOC102912116,Ppp4r2,Myadm,LOC107402623,...,EI127_mgp04,EI127_mgt17,EI127_mgt18,EI127_mgt19,EI127_mgp03,EI127_mgp02,EI127_mgt20,EI127_mgp01,EI127_mgt21,EI127_mgt22
0,LN.F1.113.2be,10,27,336,384,6,13,33,83,23,...,11886,276,122,92,6169,5420,2144,16304,173,1511
1,LN.F1.113.aa,12,35,264,340,13,7,51,36,39,...,12638,371,207,81,7024,6495,1673,16561,142,1280
2,LN.F1.113.ae,5,21,175,407,21,10,43,43,30,...,9748,272,135,67,5841,5400,1414,13018,103,955
3,LN.F1.113.ag,6,25,298,446,21,16,54,61,23,...,16001,495,230,116,9873,7522,2011,21779,196,1599
4,LN.F1.113.ak,7,25,425,526,17,19,58,132,51,...,13241,337,180,91,7773,6657,1650,16713,109,1130


In [23]:
y = df_gastroc.Animal


In [24]:
frames = [data_pca,y]

In [25]:
df_pca = pd.concat(frames, axis = 1)

In [26]:
df_pca.head()

,PC1,PC2,PC3,Animal
0,-44.117166,26.196616,6.950873,LN.F1.113.2be
1,-47.573065,-12.572077,-11.710404,LN.F1.113.aa
2,-72.231403,-6.664231,-6.881561,LN.F1.113.ae
3,17.838188,-20.307664,3.901166,LN.F1.113.ag
4,73.311040,2.032320,-22.955084,LN.F1.113.ak


In [27]:
#X = df_gastroc.drop(labels=['Animal'], axis = 1)

In [28]:
#X.head()

In [29]:
#df_animal = pd.DataFrame(y, columns=['Animal'])

In [30]:
#df_animal.head()

In [31]:
df_data = df_pca.merge(df, on='Animal', how = 'inner', indicator=True)

In [32]:

df_data.head()

,PC1,PC2,PC3,Animal,acclim1,_merge
0,-47.573065,-12.572077,-11.710404,LN.F1.113.aa,3,both
1,-72.231403,-6.664231,-6.881561,LN.F1.113.ae,3,both
2,17.838188,-20.307664,3.901166,LN.F1.113.ag,3,both
3,73.311040,2.032320,-22.955084,LN.F1.113.ak,0,both
4,50.908406,1.385255,-33.285863,LN.F1.113.am,0,both


In [54]:
X = pd.DataFrame(df_data, columns=['PC1','PC2','PC3'])
y = df_data.acclim1

In [34]:
##################################################
# Deep Learning Section ##########################
##################################################

In [55]:
X = np.array(X) 
y = np.array(y)

In [56]:
y = pd.get_dummies(y)

In [57]:
y  = y.astype("float32")

In [37]:
#input_layer_size = n                      # Dimension of features
#hidden_layer_size = input_layer_size*2    # of units in hidden layer
#output_layer_size = 4    # number of labels

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [59]:
#X_train = X_train[..., tf.newaxis].astype("float32")
#X_test = X_test[..., tf.newaxis].astype("float32")

X_train  = X_train.astype("float32")
X_test  = X_test.astype("float32")

In [60]:
X.shape

(39, 3)

In [61]:
X_train.shape

(31, 3)

In [62]:
model = Sequential()

In [63]:
model.add(Dense(128, activation='relu', input_dim=3))
model.add(Dense(4, activation='softmax'))


In [68]:
model.compile(optimizer='adam', loss = tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [70]:
model.fit(X_train, y_train, epochs=200, batch_size=10)

Epoch 1/200
4/4 [==============================] - 0s 7ms/step - loss: 0.1787 - accuracy: 0.5484
Epoch 2/200
4/4 [==============================] - 0s 11ms/step - loss: 0.1767 - accuracy: 0.5484
Epoch 3/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1842 - accuracy: 0.5161
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 0.1908 - accuracy: 0.5161
Epoch 5/200
4/4 [==============================] - 0s 15ms/step - loss: 0.2067 - accuracy: 0.4839
Epoch 6/200
4/4 [==============================] - 0s 8ms/step - loss: 0.2016 - accuracy: 0.5484
Epoch 7/200
4/4 [==============================] - 0s 11ms/step - loss: 0.1922 - accuracy: 0.5484
Epoch 8/200
4/4 [==============================] - 0s 15ms/step - loss: 0.1866 - accuracy: 0.5484
Epoch 9/200
4/4 [==============================] - 0s 11ms/step - loss: 0.1822 - accuracy: 0.5484
Epoch 10/200
4/4 [==============================] - 0s 8ms/step - loss: 0.1870 - accuracy: 0.5484
Epoch 11/200
4/4 [====